In [3]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference, LineChart
from openpyxl.chart.label import DataLabelList
from openpyxl.styles import Alignment, PatternFill, Font


def generate_excel_dashboard(input_file_path):
    # Reading data from the 'bike_buyers' sheet
    original_data = pd.read_excel(input_file_path, sheet_name='bike_buyers')
    
    # Making a copy of the raw data into a sheet called 'Working_Sheet'
    with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
        original_data.to_excel(writer, sheet_name='Working_Sheet', index=False)

    # Reading the 'Working_Sheet' data into a new dataframe
    working_data = pd.read_excel(input_file_path, sheet_name='Working_Sheet')
   
    # Data Cleaning
    working_data.drop_duplicates(keep='first', inplace=True, ignore_index=False)
    working_data['Marital Status'] = working_data['Marital Status'].replace('M', 'Married').replace('S', 'Single')
    working_data['Gender'] = working_data['Gender'].replace('F', 'Female').replace('M', 'Male')
    working_data['Age brackets'] = working_data['Age'].apply(lambda x: 'Less than 30' if x <= 30 else (
        'Greater than 55' if x > 55 else '31 to 55'))
    working_data['Commute Distance'] = working_data['Commute Distance'].replace('10+ Miles', 'More than 10 Miles')

    # Saving changes to the 'Working_Sheet'
    with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
        working_data.to_excel(writer, sheet_name='Working_Sheet', index=False)

    # Exploratory Data Analysis (EDA)
    avg_gender_income_df = np.round(
        pd.pivot_table(working_data, values='Income', index=['Gender'], columns=['Purchased Bike'],
                       aggfunc=np.mean), 2)

    with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
        avg_gender_income_df.to_excel(writer, sheet_name='Average_Gender_Income')

    wb = load_workbook(input_file_path)
    sheet1 = wb['Average_Gender_Income']

    chart1 = BarChart()
    chart1.type = "col"
    chart1.style = 10
    chart1.title = "Average Income by Gender and Purchase Data"
    chart1.y_axis.title = 'Gender'
    chart1.x_axis.title = 'Income'

    data1 = Reference(sheet1, min_col=2, min_row=1, max_row=3, max_col=3)
    cats1 = Reference(sheet1, min_col=1, min_row=2, max_row=3)
    chart1.add_data(data1, titles_from_data=True)
    chart1.dataLabels = DataLabelList()
    chart1.dataLabels.showVal = True
    chart1.set_categories(cats1)
    chart1.shape = 4
    sheet1.add_chart(chart1, "A10")
    wb.save(input_file_path)

    count_age_brackets_purchase_df = pd.pivot_table(working_data, values=['ID'], index='Age brackets',
                                                    columns='Purchased Bike', aggfunc='count')

    count_age_brackets_purchase_df = count_age_brackets_purchase_df['ID']

    with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
        count_age_brackets_purchase_df.to_excel(writer, sheet_name='Count_Age_Brackets')

    wb = load_workbook(input_file_path)
    sheet2 = wb['Count_Age_Brackets']

    chart2 = LineChart()
    chart2.style = 10
    chart2.title = "Count of Purchased with Age Brackets"
    chart2.y_axis.title = 'Count'
    chart2.x_axis.title = 'Age brackets'

    data2 = Reference(sheet2, min_col=2, min_row=1, max_row=4, max_col=3)
    cats2 = Reference(sheet2, min_col=1, min_row=2, max_row=4)
    chart2.add_data(data2, titles_from_data=True)
    chart2.dataLabels = DataLabelList()
    chart2.dataLabels.showVal = True
    chart2.set_categories(cats2)
    chart2.shape = 4
    sheet2.add_chart(chart2, "A10")
    wb.save(input_file_path)

    count_commute_distance_purchase_df = pd.pivot_table(working_data, values=['ID'], index='Commute Distance',
                                                       columns='Purchased Bike', aggfunc='count')

    count_commute_distance_purchase_df = count_commute_distance_purchase_df['ID']

    with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
        count_commute_distance_purchase_df.to_excel(writer, sheet_name='Count_Commute_Distance')

    wb = load_workbook(input_file_path)
    sheet3 = wb['Count_Commute_Distance']

    chart3 = LineChart()
    chart3.style = 10
    chart3.title = "Count of Purchased with Commute Distance"
    chart3.y_axis.title = 'Count'
    chart3.x_axis.title = 'Commute Distance'

    data3 = Reference(sheet3, min_col=2, min_row=1, max_row=6, max_col=3)
    cats3 = Reference(sheet3, min_col=1, min_row=2, max_row=6)
    chart3.add_data(data3, titles_from_data=True)
    chart3.dataLabels = DataLabelList()
    chart3.dataLabels.showVal = True
    chart3.set_categories(cats3)
    chart3.shape = 4
    sheet3.add_chart(chart3, "A10")
    wb.save(input_file_path)

    # Creating a Dashboard
    dashboard_df = pd.DataFrame()

    with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
        dashboard_df.to_excel(writer, sheet_name='Dashboard')

    wb = load_workbook(input_file_path)
    sheet_dashboard = wb['Dashboard']

    for _ in range(1, 22):
        sheet_dashboard.merge_cells('A1:R4')

    cell_dashboard = sheet_dashboard.cell(row=1, column=1)
    cell_dashboard.value = 'Bike Sales Dashboard'
    cell_dashboard.alignment = Alignment(horizontal='center', vertical='center')
    cell_dashboard.font = Font(b=True, color="F8F8F8", size=46)
    cell_dashboard.fill = PatternFill("solid", fgColor="2591DB")

    sheet_dashboard.add_chart(chart1, 'A5')
    sheet_dashboard.add_chart(chart2, 'J5')
    chart3.width = 31
    sheet_dashboard.add_chart(chart3, 'A20')
    wb.save(input_file_path)


file_path = r'C:\Users\abhil\Desktop\Excel Project Dataset.xlsx'
generate_excel_dashboard(file_path)


C:\Users\abhil\AppData\Local\Temp\ipykernel_2148\1491984312.py:36: FutureWarning: The provided callable <function mean at 0x0000021AED4198A0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.pivot_table(working_data, values='Income', index=['Gender'], columns=['Purchased Bike'],
